In [3]:
# pip install -U pip setuptools wheel

     |████████████████████████████████| 1.5 MB 1.8 MB/s eta 0:00:01
     |████████████████████████████████| 784 kB 5.7 MB/s eta 0:00:01
Requirement already up-to-date: wheel in /home/shree/anaconda3/lib/python3.7/site-packages (0.36.2)
  Attempting uninstall: pip
    Found existing installation: pip 20.0.2
    Uninstalling pip-20.0.2:
      Successfully uninstalled pip-20.0.2
  Attempting uninstall: setuptools
    Found existing installation: setuptools 45.2.0.post20200210
    Uninstalling setuptools-45.2.0.post20200210:
      Successfully uninstalled setuptools-45.2.0.post20200210
Note: you may need to restart the kernel to use updated packages.


In [4]:
# pip install -U spacy

distutils: /home/shree/anaconda3/include/python3.7m/UNKNOWN
sysconfig: /home/shree/anaconda3/include/python3.7m
user = False
home = None
root = None
prefix = None
     |████████████████████████████████| 12.8 MB 39.7 MB/s eta 0:00:01
     |████████████████████████████████| 9.8 MB 3.3 MB/s eta 0:00:01
     |████████████████████████████████| 9.1 MB 96 kB/s  eta 0:00:01
     |████████████████████████████████| 126 kB 3.4 MB/s eta 0:00:01
     |████████████████████████████████| 1.1 MB 9.8 MB/s eta 0:00:01
     |████████████████████████████████| 456 kB 15.6 MB/s eta 0:00:01
     |████████████████████████████████| 42 kB 19 kB/s s eta 0:00:01
     |████████████████████████████████| 113 kB 38.5 MB/s eta 0:00:01
     |████████████████████████████████| 82 kB 191 kB/s  eta 0:00:01
  Created wheel for smart-open: filename=smart_open-3.0.0-py3-none-any.whl size=107097 sha256=7703ced971c0b9841d3693ce32af5f6875d4f0efda52510c966f33dbdacfc747
  Stored in directory: /home/shree/.cache/pip/wheels/83/a6/12/

In [6]:
#Libraries

%matplotlib inline
import sys
import unicodedata
import re
import numpy as np
import spacy 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import *

In [9]:
# Getting the data

df = pd.read_json('content_baseline.json')
df

,Sentence,Presence
0,Help me!,1
1,Help me!,1
2,"""Front desk clerk Shawna Vela said she dialed ...",0
3,"I've been shot,' "" said Rosalinda Gonzalez, an...",0
4,Mid-Market and the Tenderloin are home to a th...,1
...,...,...
20181,"I'd flip open a cell phone, turn on its camera...",1
20182,.,0
20183,)The biggest challenge in documenting my dinin...,0
20184,"Ahumdinger TV season wrapped Wednesday night, ...",1


In [40]:
df["Sentence"][3]

'I\'ve been shot,\' " said Rosalinda Gonzalez, an assistant manager.'

# Creating the Glove Word Embeddings

In [24]:
# There are same number of 0s and 1s. I made sure the dataset is not skewed.
# We shall us SpaCy

nlp = spacy.load('en_core_web_lg')

In [25]:
len(nlp("hello").vector)

300

In [26]:
new_sentence = []

for doc in nlp.pipe(df.Sentence):
    new_sentence.append(doc)

In [28]:
print ("The token '" + str(new_sentence[3][0]) + "' has a vector of shape " + str(new_sentence[3][0].vector.shape))



The token 'I' has a vector of shape (300,)


In [34]:
sentence_array = []
for x in new_sentence[1]:
    sentence_array.append(x.vector)

In [36]:
new_sentence[1]

Help me!

In [35]:
np.asarray(sentence_array).shape

(3, 300)

So we just need to do this for every single sentence

In [37]:
glove_sentences = []

for i in tqdm(new_sentence):
    sentence_array = [np.zeros(300)]
    
    for x in i:
        if ((x.vector == np.zeros(300)).all() == False): 
            sentence_array.append(x.vector)
#     sentence_array = np.mean(sentence_array, axis = 0)
    glove_sentences.append(np.asarray(sentence_array))
        

100%|██████████| 20186/20186 [00:04<00:00, 4494.36it/s]


In [38]:
glove_sentences = np.asarray(glove_sentences)
glove_sentences.shape

/home/shree/anaconda3/lib/python3.7/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


(20186,)

Now we have the glove embeddings of our dataset.

In [33]:
np.save('glove_array.npy', glove_sentences)

# Sample Legal Text and their summaries

Text1:
    TLDR does not intend for the Sites to be used by anyone under the age of 13. You certify that you are the age of majority in your jurisdiction or are over 13 years old and using the Sites with the express consent of your parent or legal guardian. You agree not to post or contribute any personal information, such as name, address, telephone number, or email address of any person under the age of 13 to the Sites.
    
Summary1:
TLDR does not intend for the Sites to be used by anyone under the age of 13. You agree not to post or contribute any personal information, such as name, address, telephone number, or email address or email address of any person under the age of 13 to the Sites.

Text2:
If you download executable software from the Sites, the software and other materials accompanying the software (“Software”) is licensed to you by TLDR under the terms of any associated end user license agreement. TLDR does not transfer title to the Software to you. TLDR or third party licensors retain full and complete title to the Software, and all intellectual property rights embodied in it. You may not redistribute, sell, decompile, reverse engineer or disassemble the Software.

Summary2:
If you download executable software from the Sites, the software and other materials accompanying the software (“Software”) is licensed to you by TLDR under the terms of any associated end user license agreement. You may not redistribute, sell, decompile, reverse engineer or disassemble the Software.


    

In [46]:
sample_1 = [{'Sentence': 'TLDR does not intend for the Sites to be used by anyone under the age of 13.', 'Presence':1},
             {'Sentence': 'You certify that you are the age of majority in your jurisdiction or are over 13 years old and using the Sites with the express consent of your parent or legal guardian.', 'Presence':0},
             {'Sentence': 'You agree not to post or contribute any personal information, such as name, address, telephone number, or email address of any person under the age of 13 to the Sites.', 'Presence':1}]

sample_2 = [{'Sentence':'If you download executable software from the Sites, the software and other materials accompanying the software (“Software”) is licensed to you by TLDR under the terms of any associated end user license agreement.', 'Presence': 1},
           {'Sentence': 'TLDR does not transfer title to the Software to you.', 'Presence': 0},
           {'Sentence': 'TLDR or third party licensors retain full and complete title to the Software, and all intellectual property rights embodied in it.', 'Presence':0},
           {'Sentence':'You may not redistribute, sell, decompile, reverse engineer or disassemble the Software.', 'Presence':1}]


In [47]:
df1 = pd.DataFrame(sample_1)

In [48]:
df1

,Sentence,Presence
0,TLDR does not intend for the Sites to be used ...,1
1,You certify that you are the age of majority i...,0
2,You agree not to post or contribute any person...,1


In [49]:
df2 = pd.DataFrame(sample_2)

In [50]:
df2

,Sentence,Presence
0,If you download executable software from the S...,1
1,TLDR does not transfer title to the Software t...,0
2,TLDR or third party licensors retain full and ...,0
3,"You may not redistribute, sell, decompile, rev...",1


In [96]:
sentence_sample_1 = []

for doc in nlp.pipe(df1.Sentence):
    sentence_sample_1.append(doc)

In [97]:
print ("The token '" + str(sentence_sample_1[0][0]) + "' has a vector of shape " + str(sentence_sample_1[0][0].vector.shape))


The token 'TLDR' has a vector of shape (300,)


In [98]:
sentence_sample_1

[TLDR does not intend for the Sites to be used by anyone under the age of 13.,
 You certify that you are the age of majority in your jurisdiction or are over 13 years old and using the Sites with the express consent of your parent or legal guardian.,
 You agree not to post or contribute any personal information, such as name, address, telephone number, or email address of any person under the age of 13 to the Sites.]

In [99]:
glove_emb_1 = []

for x in sentence_sample_1[1]:
    glove_emb_1.append(x.vector)

In [100]:
np.asarray(glove_emb_1).shape

(33, 300)

In [101]:
glove_sentences_1 = []

for i in tqdm(sentence_sample_1):
    sentence_array = [np.zeros(300)]
    
    for x in i:
        if ((x.vector == np.zeros(300)).all() == False): 
            sentence_array.append(x.vector)
    sentence_array = np.mean(sentence_array, axis = 0)
    glove_sentences_1.append(np.asarray(sentence_array))
        

100%|██████████| 3/3 [00:00<00:00, 2920.82it/s]


In [102]:
glove_sentences_1 = np.asarray(glove_sentences_1)
glove_sentences_1.shape

(3, 300)

In [103]:
np.save('glove_array_sample_1.npy', glove_sentences_1)

In [82]:
df1.to_csv('sample_1.csv')

## Created the datasets for final testing of the summarization task with the names glove_array_sample_1.npy and glove_array_sample_2.npy